<a href="https://colab.research.google.com/github/Osowomuabe/Location-Recommender/blob/main/FDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary library
import zipfile
import os

# Define the path to your ZIP file (must be uploaded to Colab first)
zip_file_path = "/content/FraudDetectionSystem-main.zip"

# Define the extraction directory
extract_dir = "/content/FraudDetectionSystem-main"

# Check if the ZIP file exists
if os.path.exists(zip_file_path):
    # Open and extract the ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"ZIP file extracted to: {extract_dir}")
else:
    print("ZIP file not found. Please upload the file to the specified path.")


ZIP file extracted to: /content/FraudDetectionSystem-main


In [2]:
!pip install -q flask pandas scikit-learn plotly pyngrok

In [3]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv -O /content/creditcard.csv

--2025-04-15 21:54:25--  https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.207, 108.177.121.207, 209.85.145.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150828752 (144M) [text/csv]
Saving to: ‘/content/creditcard.csv’

/content/creditcard 100%[===================>] 143.84M   116MB/s    in 1.2s    

2025-04-15 21:54:26 (116 MB/s) - ‘/content/creditcard.csv’ saved [150828752/150828752]



In [4]:
from pyngrok import ngrok

NGROK_TOKEN = "2vlmV4R2NgmI6YIzIsw5udOcMLt_86iB7yUKvFwTvVbefszmu"  # Your actual token
ngrok.set_auth_token(NGROK_TOKEN)
print("✅ Ngrok ready")

✅ Ngrok ready


In [5]:
!mkdir -p templates  # Create the required templates directory
print("✅ Created templates folder")
!ls -la  # Verify the directory exists

✅ Created templates folder
total 152644
drwxr-xr-x 1 root root      4096 Apr 15 21:54 .
drwxr-xr-x 1 root root      4096 Apr 15 21:24 ..
drwxr-xr-x 4 root root      4096 Apr 11 13:37 .config
-rw-r--r-- 1 root root 150828752 Jul 29  2019 creditcard.csv
drwxr-xr-x 3 root root      4096 Apr 15 21:54 FraudDetectionSystem-main
-rw-r--r-- 1 root root   5449819 Apr 15 21:26 FraudDetectionSystem-main.zip
drwxr-xr-x 1 root root      4096 Apr 11 13:37 sample_data
drwxr-xr-x 2 root root      4096 Apr 15 21:54 templates


In [6]:
%%writefile /content/templates/dashboard.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>FraudGuard - Fraud Detection Dashboard</title>

    <!-- CSS Libraries -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap" rel="stylesheet">

    <!-- Custom Styles -->
    <style>
        :root {
            --primary-color: #1e90ff;
            --secondary-color: #ff4757;
            --success-color: #2ed573;
            --dark-color: #2f3542;
            --light-color: #f1f2f6;
        }

        body {
            font-family: 'Roboto', sans-serif;
            background-color: var(--light-color);
            color: var(--dark-color);
        }

        .navbar {
            background: linear-gradient(45deg, var(--primary-color), var(--dark-color));
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }

        .navbar-brand {
            font-weight: 700;
            font-size: 1.5rem;
        }

        .nav-link {
            font-weight: 500;
            transition: all 0.3s ease;
        }

        .nav-link:hover {
            color: var(--primary-color) !important;
            transform: translateY(-2px);
        }

        .card {
            border: none;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            transition: all 0.3s ease;
        }

        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 6px 12px rgba(0,0,0,0.15);
        }

        .card-header {
            background: linear-gradient(45deg, var(--primary-color), var(--dark-color));
            color: white;
            border-radius: 10px 10px 0 0;
        }

        .btn-primary {
            background-color: var(--primary-color);
            border: none;
            transition: all 0.3s ease;
        }

        .btn-primary:hover {
            background-color: var(--dark-color);
            transform: translateY(-2px);
        }

        .footer {
            background: var(--dark-color);
            color: white;
            padding: 20px 0;
            margin-top: 40px;
        }

        .footer a {
            color: var(--primary-color);
            text-decoration: none;
        }

        .stat-card {
            background: white;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
        }

        .stat-card h3 {
            color: var(--primary-color);
            font-weight: 700;
        }

        .chart-container {
            background: white;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 20px;
        }

        @media (max-width: 768px) {
            .navbar-brand {
                font-size: 1.2rem;
            }
        }
    </style>
</head>
<body>
    <!-- Navigation Bar -->
    <nav class="navbar navbar-expand-lg navbar-dark">
        <div class="container-fluid">
            <a class="navbar-brand" href="/">
                <i class="fas fa-shield-alt"></i> FraudGuard
            </a>
            <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav">
                <span class="navbar-toggler-icon"></span>
            </button>
            <div class="collapse navbar-collapse" id="navbarNav">
                <ul class="navbar-nav ms-auto">
                    <li class="nav-item">
                        <a class="nav-link active" href="/">Dashboard</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" href="/upload">Upload Dataset</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" href="/analysis">Analysis</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" href="/reports">Reports</a>
                    </li>
                </ul>
            </div>
        </div>
    </nav>

    <!-- Main Content -->
    <div class="container mt-4">
        <div class="row">
            <div class="col-md-3">
                <div class="stat-card">
                    <h3>Total Transactions</h3>
                    <p class="display-4">284,807</p>
                </div>
            </div>
            <div class="col-md-3">
                <div class="stat-card">
                    <h3>Fraudulent</h3>
                    <p class="display-4 text-danger">492</p>
                </div>
            </div>
            <div class="col-md-3">
                <div class="stat-card">
                    <h3>Legitimate</h3>
                    <p class="display-4 text-success">284,315</p>
                </div>
            </div>
            <div class="col-md-3">
                <div class="stat-card">
                    <h3>Accuracy</h3>
                    <p class="display-4 text-primary">99.9%</p>
                </div>
            </div>
        </div>

        <div class="row mt-4">
            <div class="col-md-12">
                <div class="chart-container">
                    <h4>Transaction Distribution</h4>
                    {% block content %}{% endblock %}
                </div>
            </div>
        </div>
    </div>

    <!-- Footer -->
    <footer class="footer text-center">
        <div class="container">
            <p class="mb-0">© 2025 FraudGuard. Developed with <i class="fas fa-heart text-danger"></i> for Secure Transactions.</p>
            <p class="mb-0">
                <a href="#">Privacy Policy</a> |
                <a href="#">Terms of Service</a>
            </p>
        </div>
    </footer>

    <!-- JavaScript Libraries -->
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</body>
</html>


Writing /content/templates/dashboard.html


In [7]:
%%writefile /content/templates/upload.html
{% extends "dashboard.html" %}

{% block content %}
<div class="row justify-content-center mt-5">
    <div class="col-md-8">
        <div class="card shadow-lg">
            <!-- Card Header -->
            <div class="card-header bg-gradient-primary text-white">
                <h3 class="card-title mb-0">
                    <i class="fas fa-upload me-2"></i>Upload Dataset
                </h3>
            </div>

            <!-- Card Body -->
            <div class="card-body">
                <form id="uploadForm" method="POST" enctype="multipart/form-data" action="/process_upload" class="needs-validation" novalidate>
                    <!-- File Input Section -->
                    <div class="mb-4">
                        <label for="datasetFile" class="form-label fw-bold">
                            Choose CSV File
                        </label>
                        <div class="input-group">
                            <input
                                class="form-control"
                                type="file"
                                id="datasetFile"
                                name="file"
                                accept=".csv"
                                required
                            >
                            <span class="input-group-text">
                                <i class="fas fa-file-csv"></i>
                            </span>
                        </div>
                        <div class="invalid-feedback">
                            Please select a valid CSV file.
                        </div>
                        <small class="form-text text-muted">
                            Supported format: .csv (Max file size: 10MB)
                        </small>
                    </div>

                    <!-- Upload Button -->
                    <div class="d-grid">
                        <button
                            type="submit"
                            class="btn btn-lg btn-success btn-block fw-bold"
                        >
                            <i class="fas fa-cloud-upload-alt me-2"></i>Upload & Analyze
                        </button>
                    </div>
                </form>

                <!-- Additional Instructions -->
                <div class="mt-4">
                    <h5 class="fw-bold">Instructions</h5>
                    <ul class="list-group list-group-flush">
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Ensure your CSV file follows the required format.
                        </li>
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Include all necessary columns (e.g., Amount, Time, Class).
                        </li>
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Avoid missing values in critical fields.
                        </li>
                    </ul>
                </div>
            </div>
        </div>
    </div>
</div>

<!-- Custom Script for Form Validation -->
<script>
(() => {
    'use strict';
    const form = document.getElementById('uploadForm');
    form.addEventListener('submit', event => {
        if (!form.checkValidity()) {
            event.preventDefault();
            event.stopPropagation();
        }
        form.classList.add('was-validated');
    }, false);
})();
</script>
{% endblock %}


Writing /content/templates/upload.html


In [8]:
%%writefile /content/templates/analysis.html
{% extends "dashboard.html" %}

{% block content %}
<div class="container-fluid mt-4">
    <!-- Page Header -->
    <div class="row mb-4">
        <div class="col-md-12">
            <h2 class="fw-bold">
                <i class="fas fa-chart-bar me-2"></i>Dataset Analysis
            </h2>
            <p class="text-muted">
                Detailed insights and statistics from the uploaded dataset.
            </p>
        </div>
    </div>

    <!-- Analysis Cards Section -->
    <div class="row g-4">
        <!-- Total Records -->
        <div class="col-md-3">
            <div class="card bg-gradient-info text-white h-100 shadow">
                <div class="card-body">
                    <h5 class="card-title">
                        <i class="fas fa-database me-2"></i>Total Records
                    </h5>
                    <p class="display-4 fw-bold">{{ total_records }}</p>
                    <small>Number of transactions analyzed</small>
                </div>
            </div>
        </div>

        <!-- Fraudulent Transactions -->
        <div class="col-md-3">
            <div class="card bg-gradient-danger text-white h-100 shadow">
                <div class="card-body">
                    <h5 class="card-title">
                        <i class="fas fa-exclamation-triangle me-2"></i>Fraudulent
                    </h5>
                    <p class="display-4 fw-bold">{{ fraud_count }}</p>
                    <small>Potential fraud cases detected</small>
                </div>
            </div>
        </div>

        <!-- Legitimate Transactions -->
        <div class="col-md-3">
            <div class="card bg-gradient-success text-white h-100 shadow">
                <div class="card-body">
                    <h5 class="card-title">
                        <i class="fas fa-check-circle me-2"></i>Legitimate
                    </h5>
                    <p class="display-4 fw-bold">{{ legit_count }}</p>
                    <small>Transactions verified as legitimate</small>
                </div>
            </div>
        </div>

        <!-- Missing Values -->
        <div class="col-md-3">
            <div class="card bg-gradient-warning text-dark h-100 shadow">
                <div class="card-body">
                    <h5 class="card-title">
                        <i class="fas fa-exclamation-circle me-2"></i>Missing Values
                    </h5>
                    <p class="display-4 fw-bold">{{ missing_values }}</p>
                    <small>Empty or incomplete records</small>
                </div>
            </div>
        </div>
    </div>

    <!-- Charts Section -->
    <div class="row mt-5">
        <div class="col-md-6">
            <div class="card shadow">
                <div class="card-header bg-gradient-primary text-white">
                    <h5 class="card-title mb-0">
                        <i class="fas fa-chart-pie me-2"></i>Transaction Distribution
                    </h5>
                </div>
                <div class="card-body">
                    <div id="pieChart"></div>
                </div>
            </div>
        </div>

        <div class="col-md-6">
            <div class="card shadow">
                <div class="card-header bg-gradient-secondary text-white">
                    <h5 class="card-title mb-0">
                        <i class="fas fa-chart-line me-2"></i>Fraud Over Time
                    </h5>
                </div>
                <div class="card-body">
                    <div id="lineChart"></div>
                </div>
            </div>
        </div>
    </div>

    <!-- Detailed Insights Section -->
    <div class="row mt-5">
        <div class="col-md-12">
            <div class="card shadow">
                <div class="card-header bg-gradient-dark text-white">
                    <h5 class="card-title mb-0">
                        <i class="fas fa-search me-2"></i>Detailed Insights
                    </h5>
                </div>
                <div class="card-body">
                    <div class="table-responsive">
                        <table class="table table-hover">
                            <thead>
                                <tr>
                                    <th>Feature</th>
                                    <th>Mean Value</th>
                                    <th>Standard Deviation</th>
                                    <th>Fraud Correlation</th>
                                </tr>
                            </thead>
                            <tbody>
                                <tr>
                                    <td>Amount</td>
                                    <td>$88.34</td>
                                    <td>250.12</td>
                                    <td>0.12</td>
                                </tr>
                                <tr>
                                    <td>Time</td>
                                    <td>94813.86</td>
                                    <td>47488.15</td>
                                    <td>0.08</td>
                                </tr>
                                <tr>
                                    <td>V1</td>
                                    <td>-0.01</td>
                                    <td>1.95</td>
                                    <td>-0.34</td>
                                </tr>
                            </tbody>
                        </table>
                    </div>
                </div>
            </div>
        </div>
    </div>
</div>

<!-- Script for Charts -->
<script>
// Initialize Pie Chart
const pieChartData = [{
    values: [{{ legit_count }}, {{ fraud_count }}],
    labels: ['Legitimate', 'Fraudulent'],
    type: 'pie',
    marker: { colors: ['#2ed573', '#ff4757'] },
    hole: 0.4
}];
Plotly.newPlot('pieChart', pieChartData, { margin: { t: 30, b: 30 } });

// Initialize Line Chart
const lineChartData = [{
    x: ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'],
    y: [12, 19, 3, 5, 2, 3],
    type: 'line',
    line: { color: '#1e90ff', width: 3 }
}];
Plotly.newPlot('lineChart', lineChartData, { margin: { t: 30, b: 30 } });
</script>
{% endblock %}


Writing /content/templates/analysis.html


In [9]:
%%writefile /content/templates/analysis.js
document.addEventListener('DOMContentLoaded', () => {
    fetch('/get_analysis')
    .then(response => response.json())
    .then(data => {
        const resultsDiv = document.getElementById('analysisResults');
        resultsDiv.innerHTML = `
            <div class="col-md-4">
                <div class="card">
                    <div class="card-body">
                        <h5 class="card-title">Records Analyzed</h5>
                        <p class="display-4">${data.total_records}</p>
                    </div>
                </div>
            </div>
            <div class="col-md-4">
                <div class="card">
                    <div class="card-body">
                        <h5 class="card-title">Fraudulent Transactions</h5>
                        <p class="display-4 text-danger">${data.fraud_count}</p>
                    </div>
                </div>
            </div>
            <div class="col-md-4">
                <div class="card">
                    <div class="card-body">
                        <h5 class="card-title">Legitimate Transactions</h5>
                        <p class="display-4 text-success">${data.legit_count}</p>
                    </div>
                </div>
            </div>`;
    });
});


Writing /content/templates/analysis.js


In [10]:
%%writefile /content/templates/reports.html
{% extends "dashboard.html" %}

{% block content %}
<div class="container-fluid mt-4">
    <!-- Page Header -->
    <div class="row mb-4">
        <div class="col-md-12">
            <h2 class="fw-bold">
                <i class="fas fa-file-download me-2"></i>Download Reports
            </h2>
            <p class="text-muted">
                Export detailed reports and data for further analysis.
            </p>
        </div>
    </div>

    <!-- Report Cards Section -->
    <div class="row g-4">
        <!-- PDF Report Card -->
        <div class="col-md-6">
            <div class="card h-100 shadow">
                <div class="card-header bg-gradient-danger text-white">
                    <h5 class="card-title mb-0">
                        <i class="fas fa-file-pdf me-2"></i>PDF Report
                    </h5>
                </div>
                <div class="card-body">
                    <p class="card-text">
                        Download a comprehensive PDF report containing:
                    </p>
                    <ul class="list-group list-group-flush">
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Summary of fraud detection results
                        </li>
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Visual charts and graphs
                        </li>
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Detailed analysis and recommendations
                        </li>
                    </ul>
                    <div class="text-center mt-4">
                        <button
                            class="btn btn-lg btn-danger fw-bold"
                            onclick="generateReport('pdf')"
                        >
                            <i class="fas fa-download me-2"></i>Download PDF
                        </button>
                    </div>
                </div>
            </div>
        </div>

        <!-- CSV Data Card -->
        <div class="col-md-6">
            <div class="card h-100 shadow">
                <div class="card-header bg-gradient-success text-white">
                    <h5 class="card-title mb-0">
                        <i class="fas fa-file-csv me-2"></i>CSV Data
                    </h5>
                </div>
                <div class="card-body">
                    <p class="card-text">
                        Export the complete dataset in CSV format for:
                    </p>
                    <ul class="list-group list-group-flush">
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Further analysis in external tools
                        </li>
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Sharing with stakeholders
                        </li>
                        <li class="list-group-item">
                            <i class="fas fa-check-circle text-success me-2"></i>
                            Archiving and record-keeping
                        </li>
                    </ul>
                    <div class="text-center mt-4">
                        <button
                            class="btn btn-lg btn-success fw-bold"
                            onclick="generateReport('csv')"
                        >
                            <i class="fas fa-download me-2"></i>Download CSV
                        </button>
                    </div>
                </div>
            </div>
        </div>
    </div>

    <!-- Additional Information Section -->
    <div class="row mt-5">
        <div class="col-md-12">
            <div class="card shadow">
                <div class="card-header bg-gradient-info text-white">
                    <h5 class="card-title mb-0">
                        <i class="fas fa-info-circle me-2"></i>Export Instructions
                    </h5>
                </div>
                <div class="card-body">
                    <div class="row">
                        <div class="col-md-6">
                            <h6 class="fw-bold">PDF Report</h6>
                            <ul class="list-unstyled">
                                <li><i class="fas fa-check me-2 text-primary"></i>Optimized for printing and sharing</li>
                                <li><i class="fas fa-check me-2 text-primary"></i>Includes visual representations of data</li>
                                <li><i class="fas fa-check me-2 text-primary"></i>Readable on all devices</li>
                            </ul>
                        </div>
                        <div class="col-md-6">
                            <h6 class="fw-bold">CSV Data</h6>
                            <ul class="list-unstyled">
                                <li><i class="fas fa-check me-2 text-primary"></i>Compatible with Excel, Google Sheets, and more</li>
                                <li><i class="fas fa-check me-2 text-primary"></i>Contains raw data for advanced analysis</li>
                                <li><i class="fas fa-check me-2 text-primary"></i>Easy to import into databases</li>
                            </ul>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>
</div>

<!-- JavaScript for Report Generation -->
<script>
function generateReport(type) {
    if (type === 'pdf') {
        alert('PDF report generation is currently in development.');
    } else if (type === 'csv') {
        alert('CSV download is currently in development.');
    }
}
</script>
{% endblock %}


Writing /content/templates/reports.html


In [11]:
!cat templates/dashboard.html  # Should show your HTML content
print("\n✅ Template created successfully")


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>FraudGuard - Fraud Detection Dashboard</title>

    <!-- CSS Libraries -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap" rel="stylesheet">

    <!-- Custom Styles -->
    <style>
        :root {
            --primary-color: #1e90ff;
            --secondary-color: #ff4757;
            --success-color: #2ed573;
            --dark-color: #2f3542;
            --light-color: #f1f2f6;
        }

        body {
            font-family: 'Roboto', sans-serif;
            background-color: var(--light-color);
            color: var(--dark-color);
        }

        .navbar {
          

In [ ]:
from flask import Flask, render_template, jsonify, request, redirect, url_for
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = '/content/uploads'
app.config['ALLOWED_EXTENSIONS'] = {'csv'}

# Ensure upload directory exists
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Load initial data
df = pd.read_csv('/content/creditcard.csv')
print(f"✅ Initial data loaded ({len(df)} transactions)")

# Prepare features
X = df.drop('Class', axis=1)
y = df['Class']
feature_list = X.columns.tolist()

# Train model
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = RandomForestClassifier(
    n_estimators=150,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)
model.fit(X_scaled, y)
print(f"✅ Model trained (Accuracy: {model.score(X_scaled, y):.2%})")

# Helper Functions
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

def analyze_dataset(filepath):
    df = pd.read_csv(filepath)
    return {
        'total_records': len(df),
        'fraud_count': df[df['Class'] == 1].shape[0],
        'legit_count': df[df['Class'] == 0].shape[0],
        'missing_values': df.isnull().sum().sum()
    }

# Routes
@app.route('/')
def home():
    fig = px.pie(
        df,
        names='Class',
        title='Transaction Distribution',
        hole=0.3,
        color_discrete_sequence=['#2ecc71', '#e74c3c']
    )
    return render_template(
        'dashboard.html',
        chart=fig.to_html(full_html=False, include_plotlyjs='cdn'),
        features=feature_list[:10]  # Show first 10 features for simplicity
    )

@app.route('/upload')
def upload():
    return render_template('upload.html')

@app.route('/process_upload', methods=['POST'])
def process_upload():
    if 'file' not in request.files:
        return redirect(url_for('upload'))

    file = request.files['file']
    if file.filename == '':
        return redirect(url_for('upload'))

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        analysis_results = analyze_dataset(filepath)
        return render_template(
            'analysis.html',
            total_records=analysis_results['total_records'],
            fraud_count=analysis_results['fraud_count'],
            legit_count=analysis_results['legit_count'],
            missing_values=analysis_results['missing_values']
        )

    return redirect(url_for('upload'))

@app.route('/analysis')
def analysis():
    return render_template('analysis.html')

@app.route('/get_analysis')
def get_analysis():
    return jsonify({
        'total_records': len(df),
        'fraud_count': df[df['Class'] == 1].shape[0],
        'legit_count': df[df['Class'] == 0].shape[0],
        'missing_values': df.isnull().sum().sum()
    })

@app.route('/reports')
def reports():
    return render_template('reports.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        input_values = np.array([[data[f] for f in feature_list]])
        scaled_input = scaler.transform(input_values)

        proba = model.predict_proba(scaled_input)[0]
        return jsonify({
            "prediction": int(model.predict(scaled_input)[0]),
            "confidence": float(max(proba)),
            "probabilities": {
                "legitimate": float(proba[0]),
                "fraud": float(proba[1])
            }
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 400

# Run with Ngrok
if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"\n🔗 Application URL: {public_url}\n")
    app.run(port=5000)


✅ Initial data loaded (284807 transactions)
✅ Model trained (Accuracy: 99.97%)

🔗 Application URL: https://ddda-34-44-211-210.ngrok-free.app

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 22:00:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 22:01:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 22:01:19] "GET /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 22:01:55] "GET /analysis HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 22:02:23] "GET /reports HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 22:03:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 22:11:23] "GET /upload HTTP/1.1" 200 -
ERROR:__main__:Exception on /process_upload [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
          